# Lab 2 Key: Data Wrangling and Exploratory Data Analysis
Now that you have the basics down let's start looking at some real data! After completing this lab you will be able to:
- Import data into a pandas `Data Frame`
- Read and use the pandas documentation to implement a function
- Merge datasets using common fields
- Filter data
- Calculate new columns
- Transform data from wide to long and long to wide
- Create some simple charts
- Sort data

Some if this should look familiar after completing lab number 1. Refer back the the previous lab as well as to the pandas [cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) or [documentation](https://pandas.pydata.org/docs/index.html) as you run into trouble. 

For this lab we will be using UDOT pavement data from:
- Sections Data Source: https://data-uplan.opendata.arcgis.com/datasets/historic-pavement-section-data-open-data/
- Attributes Data Source: https://maps.udot.utah.gov/arcgis/rest/services/Complex/PAVE_PaveMgmtLevelHistory/MapServer/1

In [ ]:
### Only run this code if using Jupyter Lite ###
#Installs plotting library
%pip install -q seaborn

In [ ]:
import pandas as pd

#Numpy is a package with additional math and data functions
import numpy as np

#Matplotlib and Seaborn are data visualizaiton packages that we will use for plotting down the line
import matplotlib.pyplot as plt
import seaborn as sns

## Reading and Merging the Data
First we need to read the data as a dataframe. Here is an example reading in the sections data, look at then then try reading in the attribute table yourself! You'll also see us use the `.head()` method which displays the first n rows of the data frame (with five rows being the default). **Hint:** `.tail()` will display the last n rows of a data frame (with 5 as the default)

In [ ]:
#Import sections dataset from the relative file path (the pavement_section_data.csv file located in the data folder)
#Note Filepath is different from primary lab
sections = pd.read_csv('../data/pavement_section_data.csv')

#Display the first 5 rows of `sections`
sections.head()

<div class="alert alert-block alert-info"><b>Exercise:</b> Import the attributes dataset located at <code>'data/pavement_attributes.csv'</code> into a variable named <code>attrs</code> and display the first five rows</div>

In [ ]:
#Note: Filepath is different from primary lab
attrs = pd.read_csv('../data/pavement_attributes.csv') ### Your Code Here ###
attrs.head()

Now that we have our data frames we need to combine them into one data frame with both sets of columns. We can do this using the `pd.merge()` function (think of it like a SQL `JOIN` or an Excel `VLOOKUP`). To get started with this function take a look at its [documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html). While reading the documentation for a package can appear daunting at first it often tells you much that you need to know to implement a new tool. Typically you can see the function name and the arguments it accepts as well as the defaults for each argument. As you scroll down the page you can see a more in depth description of each argument and the options available for it as well as some examples. 

<div class="alert alert-block alert-info"><b>Exercise:</b> Create a new variable called <code>df</code> using the <code>pd.merge()</code>function that uses <code>sections</code> as the left dataframe and <code>attrs</code> as the right. Use the list of common keys defined below as the keys to match on. Use the documentation and examples for help.</div>

In [ ]:
#A list of key collumns included in both datarames
common_keys = ["ROUTE", "DIR", "LOCATION", "REGION"]

df = pd.merge(sections, attrs, on=common_keys)

# Display the head of the Dataframe
df.head()

If you're having trouble click the drop down arrow to see the answer:
<details>
    <summary>Click here to hide/unhide the answer!</summary>
  
  ### Answer
  ```python
df = pd.merge(sections, attrs, on=common_keys)
  ```

Now that we have the data joined together, let's take a look at the data and do some initial exploration of the data! In the last lab you filtered a data frame, selected specific columns, and got counts of the unique values in individual columns. Let's put those skills to use! Look at the previous lab as a reference and click the hint arrow below as a last resort!

<div class="alert alert-block alert-info"><b>Exercise:</b> This is a multi part exercise with a few steps:
    <ol>
        <li>Get the <code>.value_counts()</code> of the 'SURFACE_TY' column</li>
        <li>You should see three records with 'Gravel' as the surface type. Filter for these rows using a condition (<code>'SURFACE_TY' == 'Gravel'</code>) so we can take a closer look</li>
        <li>There are a lot of columns to look at so lets look at a smaller selection of columns, specifically <code>'ROUTE'</code>, <code>'LOCATION'</code>, <code>'SURFACE_TY'</code>, and <code>'AADT_2011'</code>. <b>Hint:</b> Think first about filtering the rows, then select the columns you are looking for.</li>
    </ol>
</div>

In [ ]:
df['SURFACE_TY'].value_counts() ### Get the count of unique surface types ###

In [ ]:
df[df['SURFACE_TY']=='Gravel']### Filter for only records where the surface type is 'Gravel' ###

In [ ]:
df[df['SURFACE_TY']=='Gravel'][['ROUTE', 'LOCATION', 'SURFACE_TY', 'AADT_2011']]### Filter again, then show only the Route, Location, Surface Type, and 2011 AADT fields ###

If you're having trouble click the drop down arrow to see the answers:
<details>
    <summary>Click here to hide/unhide the answers!</summary>
  
  ### Answers
  ```python
    df['SURFACE_TY'].value_counts()   #Get the count of each type of surface
    df[df['SURFACE_TY']=='Gravel']    #Show only records with 'Gravel' as the Surface Type
    df[df['SURFACE_TY']=='Gravel'][['ROUTE', 'LOCATION', 'SURFACE_TY', 'AADT_2011']] #Show only the selected columns for records where the surface type is gravel
  ```

## Cleaning the Data
Now we need to a little data processing. First things first we need to filter the data so we only use the rows we are interested in looking at. For our purposes we want rows where the surface type is not Gravel that were overlaid (`'YR_OVL'`) before 2013. We've already explored filtering above but in this case we want to filter and overwrite our data frame so df only includes the rows we are interested. Here is an example for removing the segments with gravel as the surface type:

In [ ]:
#Reset the variable df to be equal to df where df['SURFACE_TY'] is not equal to 'Gravel'
df = df[df['SURFACE_TY']!='Gravel']

<div class="alert alert-block alert-info"><b>Exercise:</b> Your turn! Filter <code>df</code> so that we only keep rows where the <code>'YR_OVL'</code> is before 2013. Take a look at the comparison operators cheat sheet if you need to!</div>

In [ ]:
### Your Code Here ###
df = df[df['YR_OVL']<2013]

Next we want to calculate a few columns. Specifically we need a column with a unique ID for each row, and a column with the difference between the year overlaid (`'YR_OVL'`) and year surfaced (`'YR_SURF'`). 

To create a unique column id we will join the route, direction, and object id fields by applying python's `str.join()` method. To do this we need to first make sure all of the columns are strings, then we can use the pandas `DataFrame.apply()` method to join them together. Let's start by looking at the data types of each column:

In [ ]:
#Create a list of the id columns we want to use
id_cols = ['ROUTE', 'DIR', 'OBJECTID']
#Get the info for these columns
df[id_cols].info()

It looks like `ROUTE` and `DIR` are already objects (string) data but `OBJECTID` is not. Let's fix that using `.astype(str)`

In [ ]:
#Rewrite the object id column with the object id column cast to a string
df['OBJECTID'] = df['OBJECTID'].astype(str)

Now that our columns are all objects we can join them. We use the pandas `.apply()` method (which allows you to apply a function along every row of a data frame), and the python `str.join()` method (which concatenates a list of strings using a seperator, in this case, `'-'`).

In [ ]:
#Create a new column called id that inlcludes the joined id columns
df['id'] = df[id_cols].apply('-'.join, axis=1)

Now for something a little simpler, we want the difference between two columns, You've done something like this in the last lab so try to apply that knowledge here.

<div class="alert alert-block alert-info"><b>Exercise:</b> Create a new column of the data frame called <code>'gap'</code> that is the difference between the <code>'YR_OVL'</code> and <code>'YR_SURF'</code>.</div>

In [ ]:
df['gap'] = df['YR_SURF'] - df['YR_OVL'] ### Your Code Here ###

## Data Transformations

Often, before we can plot or model our data, we need to ensure that our data is in a proper format. This can mean many things depending on our needs, but some of the most common transformations include shifting data from wide to long.

<figure style="text-align: center;">
  <img src="../links/wide-long.png" alt="A graphic displaying data in wide vs long formats" width="600" style="display: block; margin: 0 auto;">
  <figcaption style="max-width: 80%; margin: 0 auto;">Wide data includes a record (row) for every unique item, with observations recorded in separate columns. Long data includes a record for every observation with a single column for the type of observation. Source: <a href="https://louisahsmith.github.io/R-office-hours/OH-10.html">https://louisahsmith.github.io/R-office-hours/OH-10.html</a></figcaption>
</figure>

Long data is often used in data science to look not just at the object being recorded but each metric or observation individually. Often it is the format required for generating certain types of plot or allows for more effective time series analysis. In python we can use the `pandas.melt()` function to transform our data. We do this by identifying the columns to use as ids (the columns we would like to keep without transformation) and value variables (the columns containing each observation that we would like to transform from wide to long).

In [ ]:
#Define the id variables of each record to keep after pivoting.
id_vars = ['id', 'SURFACE_TY', 'REGION', 'YR_SURF', 'CLASS', 'MILES', 'AADT_2011']

#Define the list of values we would like to unpivot
value_vars = ['OCI_04', 'OCI_05', 'OCI_06', 'OCI_07', 'OCI_08', 'OCI_09', 'OCI_10', 'OCI_11', 'OCI_12']

### Create and save a long data frame to a new variable  ###
long_df = pd.melt(             #For longer functions it can be helpful to break it up on multiple lines
    df,
    id_vars = id_vars,         #set id_vars keyword to the defined id_vars list we created above
    value_vars = value_vars,   #set value_vars keyword to the defined value_vars list we created above
    var_name =   'obs_year',   #set the column title for the column defining the variable 
    value_name = 'OCI'         #set the column title for the value column
)

#Display the head of long_df
long_df.head()

Let's hold on to this new long data frame and come back to it after discussing exploratory data analysis.

## Exploratory Data Analysis

Now we that we have the data we need in clean and useful format let's begin to explore a bit more about it. You've already been able to see the head and tail of a data frame, as well as the count of unique values.

In [ ]:
#See a list of all of the columns in our data frame
df.columns

In [ ]:
#See some summary statistics about each column in our dataframe
df.describe()

In [ ]:
#Get a count of the records for each column by class and region
df.groupby(['CLASS', 'REGION']).size()

This is the pandas `DataFrame.groupby()` method. If you are familiar with SQL `GROUPBY` functions or Excel pivot tables this works in a similar way. It allows you to aggregate the data based on one or more variables and perform a variety of calculations such as `.sum()`, `.mean()`, `.count()`, and more! You can even perform different calculations on different columns. If you'd like to look at the more advanced options, look at the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby).

It is also often helpful to see the distribution of and relationships in data by doing simple visualizations. We'll cover more advanced data visualiztion in lab 3 but for now lets start with the built in histogram functionality in pandas using the `.hist()` method.

In [ ]:
#Select the OCI_12 column and create a histogram with 10 bins.
df['OCI_12'].hist(bins=10)

## Your Turn
Now it is your turn to do some exploratory data analysis and data cleaning! Go through the series of exercises below on your own to explore the data frames we have created and do some additional processing on them. Let's go back a step and start with our first data frame, `df`

<div class="alert alert-block alert-info"><b>Exercise:</b> Use the pandas <code>.groupby()</code> method to aggregate the data in <code>df</code>. 
    <ul>Get:
        <li>A count of all segments by surface type (SURFACE_TY) and region (REGION)</li>
        <li>The sum of miles (MILES) by the overlay year (YR_OVL) <b>Hint:</b> think about how you select a single column from the data frame and apply that after you sum</li>
        <li>The sum of miles by the year surfaced (YR_SURF)</li>
    </ul>
</div>

In [ ]:
df.groupby(['SURFACE_TY', 'REGION']).size()### Get the count of all segments by surface types and region ###

In [ ]:
df.groupby('YR_OVL').sum()['MILES']### Get the sum of miles by the overlay year ###

In [ ]:
df.groupby('YR_SURF').sum()['MILES'] ### Get the sum of miles by the year surfaced

<div class="alert alert-block alert-info"><b>Exercise:</b> Create histograms of the following columns: 
    <ul>
        <li>RIDE_12</li>
        <li>RUT_12</li>
        <li>FALT_12</li>
    </ul>
</div>

In [ ]:
df['RIDE_12'].hist()### Your Code Here ###

In [ ]:
df['RUT_12'].hist()### Your Code Here ###

In [ ]:
df['FALT_12'].hist()### Your Code Here ###

We also often want to see how correlated our data is. This is a little more complicated to plot out but take a look at the possiblities and come back to this after you've done Lab 3 to put together what is happening here!

In [ ]:
# This plots a correlation matrix for the dataframe.

#Choose the variables to check correlation
vars = ['OCI_12', 'RIDE_12', 'RUT_12', 'FALT_12']

#Set the Figure Size to 5 wide and 5 tall
fig, ax = plt.subplots(figsize=(5, 5)) 

#Create a correlation matrix of vars
corr_matrix = df[vars].corr() 

#View correlation matrix as a seaborn heatmap
sns.heatmap(corr_matrix, cmap='RdBu', annot=True, ax=ax) 

#Display the plot
plt.show()

Now do some exploratory analysis with `long_df`. Let's examine the `OCI` column using `.describe()` and `.hist()`

<div class="alert alert-block alert-info"><b>Exercise:</b> Examine <code>long_df</code> using histograms and descriptive statistics by looking specifically at the <code>'OCI'</code> column.
</div>

In [ ]:
### Your Code Here ###
long_df['OCI'].describe()

In [ ]:
### Your Code Here ###
long_df['OCI'].hist()

You'll notice that the lowest values of OCI are negative one, which isn't a valid observation. We can presume these were used to record that the data is missing so we can filter out these rows.
<div class="alert alert-block alert-info"><b>Exercise: </b>How many rows of <code>long_df</code> have an OCI value of -1? (Note: This is good practice before dropping rows) Then remove the rows of long_df where OCI is equal to -1.
</div>

In [ ]:
### Your Code Here ###
len(long_df[long_df['OCI']==-1])

In [ ]:
### Your Code Here ###
long_df = long_df[long_df['OCI']!=-1]

In [ ]:
### Check your Work ###
assert (long_df['OCI']>=0).all()

Next create a new column with the year of the OCI observation. There are a number of ways you could do this but we are going to follow these steps:
- Use string slicing to get the last two characters of each record (this will look like `your_column.str[-2:]`)
- Cast this string to an integer data type. **Hint:** you can chain this on top of the step before.
- Add 2000 to the result.

<div class="alert alert-block alert-info"><b>Exercise:</b> Create a new <code>'year'</code> column in <code>long_df</code> using the steps outlined above.
</div>

In [ ]:
long_df['year'] = long_df['obs_year'].str[-2:].astype(int)+2000 ### Your Code Here ###

In [ ]:
#Check that the column was created as expected
assert 1200 < len(long_df[long_df['year'] == 2005]['year']) < 1250

Now that we have a year column, let's examine the distribution of OCI by year and the number of segments surfaced in each year.

In [ ]:
#Create boxplot of OCI
long_df.boxplot(column='OCI', by='year')

In [ ]:
#Create barplot of the count of segments from df. Sorting the index assures the years are plotted in order.
df['YR_SURF'].value_counts().sort_index().plot(kind='bar')

The last thing we need to do before the bonus exercise is to sort the long data frame and save it.

In [ ]:
#Sort long_df by id and year and save it inplace (save the sorted version over long_df)
long_df.sort_values(by=['id', 'year'], inplace=True)

#Save long_df as a csv to read in later
long_df.to_csv('../data/long.csv')

## Bonus Exercise
Try a challenge! Let's find the number of total miles by region and class then make it a little nicer by pivoting it to a wide table. You'll need to follow a few steps:
- Group by class and region to get the sum of miles. (You'll need to add `.reset_index()` at the end for the next step to work)
- Use the pandas `DataFrame.pivot()` method. Look at the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) for the method to see how it works.

You want your final output to look like this:
<img src="links/Pivot example.png" alt="An axample of a pivoted data frame" width="200" style="display: block; margin: 0 auto;">

In [ ]:
### Your Code Here ###

summary = df.groupby(['CLASS', 'REGION']).sum()['MILES'].reset_index()

summary.pivot(index='CLASS', columns='REGION', values='MILES')